<a href="https://colab.research.google.com/github/OlesyaValukhova/walkability/blob/main/%D0%94%D0%BE%D1%81%D1%82%D1%83%D0%BF%D0%BD%D0%BE%D1%81%D1%82%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install osmium osmnx h3pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.7 MB/s eta 0:00:00


In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import folium
import osmnx as ox
import h3pandas
import osmium as osm
from shapely.geometry import Polygon
from shapely.geometry import Point
from shapely.ops import unary_union
from shapely.ops import nearest_points
from folium import plugins

In [5]:
# @title
#база

# Получение данных по городу из OSM
place_name = 'Пятигорск'
gdf = ox.geocode_to_gdf(place_name)
# Создание карты
m = folium.Map([gdf.centroid.y, gdf.centroid.x], tiles='cartodbpositron', width=1200, height=700, zoom_start=12)
# на карту добавляем полигон, построенный по координатам краев города
folium.GeoJson(gdf).add_to(m)
folium.FitBounds([[gdf.bounds.miny[0], gdf.bounds.minx[0]],[gdf.bounds.maxy[0], gdf.bounds.maxx[0]]]).add_to(m)
# Группа слоев для гексагонов с зеленью

green_layer = folium.FeatureGroup(name='Интенсивность зелени')
transport_layer = folium.FeatureGroup(name='Общественный транспорт')
food_layer = folium.FeatureGroup(name='Точки питания')
mineral_water_layer = folium.FeatureGroup(name='Бюветы')
leisure_layer = folium.FeatureGroup(name='Досуг')
shop_layer = folium.FeatureGroup(name='Магазины')
resort_layer = folium.FeatureGroup(name='Санатории')


# Построение гексагонов с зеленью
tags = {
    'natural': True,
    'leisure': ['park', 'garden', 'dog_park', 'nature_reserve'],
    'landuse': ['meadow', 'forest']
}
ptg_hex = gdf.h3.polyfill_resample(9)


green = ox.features.features_from_polygon(gdf.geometry.values[0], tags)
green_geojson = folium.features.GeoJson(green)
gdf_osm = (green)
gdf_hexagons = ptg_hex


gdf_hexagons = ptg_hex  # Загрузка данных гексагонов
gdf_hexagons['weighted_intersections'] = gdf_hexagons.apply(
    lambda hexagon: sum(4 if row['natural'] == 'wood' else 1
                        for _, row in gdf_osm[gdf_osm.intersects(hexagon.geometry)].iterrows()),
    axis=1)
def get_color(x):
    # Пример условия для определения цвета
    if x == 0:
        return 'white'
    elif x in [1,4]:
        return 'PaleGreen'
    elif x in [5, 8]:
        return 'MediumSeaGreen'
    else:
        return 'darkgreen'
# Добавление гексагонов на карту и в группу слоев
for _, row in gdf_hexagons.iterrows():
    color = get_color(row['weighted_intersections'])
    folium.Polygon(
        locations=[(lat, lon) for lon, lat in list(row.geometry.exterior.coords)],
        color=color,
        fill=True,
        fill_color=color
    ).add_to(green_layer)
# Добавление групп слоев на карту
green_layer.add_to(m)

# Загрузка данных о санаториях
defining_points_tags = {
  "leisure": ["resort"]
}
defining_points = ox.features.features_from_polygon(gdf.geometry.values[0], defining_points_tags)
def_points_gdf = gpd.GeoDataFrame(defining_points)
folium.GeoJson(
    defining_points,
    style_function=lambda x: {'color': 'red'}
 ).add_to(resort_layer)
for index, stop in def_points_gdf.iterrows():
    circle = Point(stop.geometry.centroid.x, stop.geometry.centroid.y).buffer(0.01)
    intersected = def_points_gdf[def_points_gdf.geometry.intersects(circle)]
    folium.Circle(
        location=[stop.geometry.centroid.y, stop.geometry.centroid.x],
        radius=1000,
        color='red',
        fill=False,
        fill_color='red',
        fill_opacity=0,  # Прозрачность заливки (для сохранения постоянства)
        opacity=0.9,
    ).add_to(resort_layer)
resort_layer.add_to(m)


#транспорт
t_tags = {
    "highway":["bus_stop"],
    "public_transport":["stop_position"]
    }
transport = ox.features.features_from_polygon(gdf.geometry.values[0], t_tags)
transport_gdf= gpd.GeoDataFrame(transport)
circles = []
for index, stop in transport_gdf.iterrows():
    circle = Point(stop.geometry.centroid.x, stop.geometry.centroid.y).buffer(0.01)
    circles.append(circle)
# Объединение всех окружностей в одну геометрию
combined_geometry = unary_union(circles)
# Рисование полигона на карте, соответствующего объединенной геометрии
folium.GeoJson(
    combined_geometry,
    style_function=lambda x: {'fillColor': 'purple', 'color': 'purple', 'fillOpacity': 0.2}
).add_to(transport_layer)

transport_layer.add_to(m)


#бюветы
w_tags = {"water_characteristic":["mineral", "sulfuric"]}
water = ox.features.features_from_polygon(gdf.geometry.values[0], w_tags)
w_gdf = gpd.GeoDataFrame(water)
w_circles = []
for index, stop in w_gdf.iterrows():
    circle = Point(stop.geometry.centroid.x, stop.geometry.centroid.y).buffer(0.01)
    w_circles.append(circle)
combined_geometry = unary_union(w_circles)
folium.GeoJson(
    combined_geometry,
    style_function=lambda x: {'fillColor': 'white', 'color': 'white', 'fillOpacity': 0.55}
).add_to(mineral_water_layer)

mineral_water_layer.add_to(m)


#питание
food_tags = {
  "amenity":['fast_food', 'cafe', 'restaurant', 'food_court', 'ice_cream'],
}
food = ox.features.features_from_polygon(gdf.geometry.values[0], food_tags)
food_gdf = gpd.GeoDataFrame(food)
food_circles = []
for index, stop in food_gdf.iterrows():
    circle = Point(stop.geometry.centroid.x, stop.geometry.centroid.y).buffer(0.01)
    food_circles.append(circle)
combined_geometry = unary_union(food_circles)
folium.GeoJson(
    combined_geometry,
    style_function=lambda x: {'fillColor': 'yellow', 'color': 'yellow', 'fillOpacity': 0.55}
).add_to(food_layer)
food_layer.add_to(m)


#досуг
les_tags = {
    'tourism':['artwork', 'attraction', 'gallery', 'museum', 'picnic_site', 'theme_park', 'viewpoint', 'zoo'],
    'amenity':['arts_centre', 'cinema,', 'exhibition_centre', 'fountain', 'music_venue', 'nightclub', 'planetarium', 'theatre', '	public_bath']
}
leisure = ox.features.features_from_polygon(gdf.geometry.values[0], les_tags)
les_gdf = gpd.GeoDataFrame(leisure)

for index, stop in les_gdf.iterrows():
    circle = Point(stop.geometry.centroid.x, stop.geometry.centroid.y).buffer(0.01)
    intersected = les_gdf[les_gdf.geometry.intersects(circle)]
    folium.Circle(
        location=[stop.geometry.centroid.y, stop.geometry.centroid.x],
        radius=1000,
        color='pink',
        fill=True,
        fill_color=False,
        fill_opacity=0.02,  # Прозрачность заливки (для сохранения постоянства)
        opacity=0.6,
    ).add_to(leisure_layer)
leisure_layer.add_to(m)


#магазины
shop_tags = {
  "shop":True,
}
shop = ox.features.features_from_polygon(gdf.geometry.values[0], shop_tags)
shop_gdf = gpd.GeoDataFrame(shop)
shop_circles = []
# for index, stop in shop_gdf.iterrows():
#     circle = Point(stop.geometry.centroid.x, stop.geometry.centroid.y).buffer(0.01)
#     shop_circles.append(circle)
# combined_geometry = unary_union(shop_circles)
# folium.GeoJson(
#     combined_geometry,
#     style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'fillOpacity': 0.1}
# ).add_to(shop_layer)
# shop_layer.add_to(m)



# Модифицируем код для отображения магазинов с учетом доступности к санаториям и остановкам общественного транспорта
for _, row in shop_gdf.iterrows():
    # Находим ближайший санаторий
    nearest_sanatorium = def_points_gdf.geometry.distance(row.geometry).idxmin()
    # Находим ближайшую остановку общественного транспорта
    nearest_transport_stop = transport_gdf.geometry.distance(row.geometry).idxmin()

    # Рассчитываем расстояние до ближайшего санатория и остановки общественного транспорта
    distance_to_sanatorium = row.geometry.distance(def_points_gdf.loc[nearest_sanatorium].geometry)
    distance_to_transport_stop = row.geometry.distance(transport_gdf.loc[nearest_transport_stop].geometry)

    # Если расстояние до ближайшего санатория или остановки меньше порогового значения, отмечаем магазин желтым цветом
    if distance_to_sanatorium < 1000 or distance_to_transport_stop < 1000:
        folium.Circle(
            location=[row.geometry.centroid.y, row.geometry.centroid.x],
            radius=10,
            color='yellow',
            fill=True,
            fill_color='yellow',
            fill_opacity=0.6
        ).add_to(shop_layer)
    else:
        # В остальных случаях отмечаем магазин синим цветом
        folium.Circle(
            location=[row.geometry.centroid.y, row.geometry.centroid.x],
            radius=10,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(shop_layer)

# Добавляем слой магазинов на карту
shop_layer.add_to(m)

# Добавление Layer Control на карту
folium.LayerControl().add_to(m)
# Отображение карты
m



<ipython-input-5-993991834>:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map([gdf.centroid.y, gdf.centroid.x], tiles='cartodbpositron', width=1200, height=700, zoom_start=12)
/usr/local/lib/python3.11/dist-packages/folium/utilities.py:103: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/usr/local/lib/python3.11/dist-packages/folium/utilities.py:109: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/usr/local/lib/python3.11/dist-packages/folium/utilities.py:111: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

In [6]:
gdf = ox.geocode_to_gdf('Пятигорск')
hm = folium.Map([gdf.centroid.y, gdf.centroid.x], tiles='cartodbpositron', width=1500, height=1000, zoom_start=12)
# Группа слоев для тепловой карты
folium.GeoJson(gdf, style_function=lambda x: {'color': 'blue', 'fillOpacity': 0.001}).add_to(hm)
folium.FitBounds([[gdf.bounds.miny[0], gdf.bounds.minx[0]],[gdf.bounds.maxy[0], gdf.bounds.maxx[0]]]).add_to(hm)
heat_layer = folium.FeatureGroup(name='Тепловая карта')
all_points = []
# Собираем точки из слоя с зеленью
for _, row in gdf_osm.iterrows():
    point = row.geometry.centroid
    all_points.append((point.y, point.x))

# Собираем точки из слоя с общественным транспортом
for _, row in transport_gdf.iterrows():
    point = row.geometry.centroid
    all_points.append((point.y, point.x))

# Собираем точки из слоя с бюветами
for _, row in w_gdf.iterrows():
    point = row.geometry.centroid
    all_points.append((point.y, point.x))

# Собираем точки из слоя с питанием
for _, row in food_gdf.iterrows():
    point = row.geometry.centroid
    all_points.append((point.y, point.x))

# Собираем точки из слоя с досугом
for _, row in les_gdf.iterrows():
    point = row.geometry.centroid
    all_points.append((point.y, point.x))

# Собираем точки из слоя с магазинами
for _, row in shop_gdf.iterrows():
    point = row.geometry.centroid
    all_points.append((point.y, point.x))

# Создание тепловой карты
plugins.HeatMap(all_points, gradient={0.3 : 'Azure', 0.5 :'LightCyan',  0.6: 'PowderBlue', 0.8: 'DeepSkyBlue', 1:'Fuchsia'}).add_to(heat_layer).add_to(heat_layer)

heat_layer.add_to(hm)

# Добавление Layer Control на карту
folium.LayerControl().add_to(hm)

# Отображение карты
hm

<ipython-input-6-1104851910>:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hm = folium.Map([gdf.centroid.y, gdf.centroid.x], tiles='cartodbpositron', width=1500, height=1000, zoom_start=12)
/usr/local/lib/python3.11/dist-packages/folium/utilities.py:103: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/usr/local/lib/python3.11/dist-packages/folium/utilities.py:109: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/usr/local/lib/python3.11/dist-packages/folium/utilities.py:111: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) inst